### Talk Time

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

In [ ]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column row result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"search_string",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [ ]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [ ]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [ ]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [ ]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/1-Talk Time/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
df_word_select

In [ ]:
df_word_select.to_excel(f"{lang_folder.capitalize()}_200_Word.xlsx", index=False)

In [ ]:
word_list = df_word_select["word"].to_list()
len(word_list)

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
df_link_default = pd.DataFrame(data=[["repeat",0,2,"bj1JRuyYeco","https://www.youtube.com/watch?v=bj1JRuyYeco&t=0s"]], columns=["search_string","start_time","end_time","video_id","video_url"])
df_link_default

In [ ]:
df_word_link = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/1-Talk Time Data Prepare/{lang_folder.capitalize()}/{lang_folder.capitalize()}_200_Word_6_Youtube_0.6s_Timeshift_For_Talk_Time_Result_Manuel.xlsx")
df_word_link

In [ ]:
df_word_link[df_word_link["search_string"].duplicated()]

In [ ]:
df_twogram_link = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/1-Talk Time Data Prepare/{lang_folder.capitalize()}/{lang_folder.capitalize()}_Twogram_With_200_Word_6_Youtube_0.6s_Timeshift_For_Talk_Time_Result_Manuel.xlsx")
df_twogram_link

In [ ]:
df_threegram_link = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/1-Talk Time Data Prepare/{lang_folder.capitalize()}/{lang_folder.capitalize()}_Threegram_With_200_Word_6_Youtube_0.6s_Timeshift_For_Talk_Time_Result_Manuel.xlsx")
df_threegram_link

In [ ]:
df_sentence_link = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/1-Talk Time Data Prepare/{lang_folder.capitalize()}/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Sample_Selected_Manuel.xlsx")
df_sentence_link

In [ ]:
# used for multi search result
#twogram_link_list = df_twogram_link["search_string"].to_list()
#threegram_link_list = df_threegram_link["search_string"].to_list()
df_result = pd.DataFrame()
for sent in df_sentence_link["search_string"]:

    # words
    sent_words = word_tokenize(sent)

    # twogram
    twogram_zip = ngrams(sent.split(), 2)
    twogram_list = [" ".join(x) for x in twogram_zip]
    #df_twogram_var = pd.DataFrame(data=twogram, columns=["twogram"])
    
    # threegram
    threegram_zip = ngrams(sent.split(), 3)
    threegram_list = [" ".join(y) for y in threegram_zip]
    #df_threegram_var = pd.DataFrame(data=threegram, columns=["threegram"])
    
    # word result
    for word in sent_words:        
        df_word_search_var = df_word_link[df_word_link["search_string"] == word]
        df_word_search_var.reset_index(drop=True, inplace=True)
        for i in range(len(df_word_search_var)):
            df_link_default_var = df_link_default
            try:
                word_time_diff_var = df_word_search_var.loc[i,"end_time"] - df_word_search_var.loc[i,"start_time"]
                df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var+1.0
                df_result = pd.concat([df_result,df_word_search_var.iloc[[i,]]], axis=0)
                df_result = pd.concat([df_result,df_link_default_var], axis=0)
            except:
                pass

    # twogram result
    df_twogram_search_var = df_twogram_link[df_twogram_link["search_string"].isin(twogram_list)]
    df_twogram_search_var.reset_index(drop=True, inplace=True)
    for j in range(len(df_twogram_search_var)):
        df_link_default_var = df_link_default
        try:
            twogram_time_diff_var = df_twogram_search_var.loc[j,"end_time"] - df_twogram_search_var.loc[j,"start_time"]
            df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + twogram_time_diff_var+1.0
            df_result = pd.concat([df_result,df_twogram_search_var.iloc[[j,]]], axis=0)
            df_result = pd.concat([df_result,df_link_default_var], axis=0)
        except:
            pass

    # threegram result
    df_threegram_search_var = df_threegram_link[df_threegram_link["search_string"].isin(threegram_list)]
    df_threegram_search_var.reset_index(drop=True, inplace=True)
    for k in range(len(df_threegram_search_var)):
        df_link_default_var = df_link_default
        try:
            threegram_time_diff_var = df_threegram_search_var.loc[k,"end_time"] - df_threegram_search_var.loc[k,"start_time"]
            df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + threegram_time_diff_var+1.0
            df_result = pd.concat([df_result,df_threegram_search_var.iloc[[k,]]], axis=0)
            df_result = pd.concat([df_result,df_link_default_var], axis=0)
        except:
            pass

    # sentence added
    df_sent_search_var = df_sentence_link[df_sentence_link["search_string"] == sent]
    df_sent_search_var.reset_index(drop=True, inplace=True)
    for l in range(len(df_sent_search_var)):
        df_link_default_var = df_link_default
        try:
            sent_time_diff_var = df_sent_search_var.loc[l,"end_time"] - df_sent_search_var.loc[l,"start_time"]
            df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + sent_time_diff_var+1.0
            df_result = pd.concat([df_result,df_sent_search_var.iloc[[l,]]], axis=0)
            df_result = pd.concat([df_result,df_link_default_var], axis=0)
        except:
            pass

df_result.reset_index(drop=True, inplace=True)   

In [ ]:
df_result

In [ ]:
df_result[df_result["search_string"] == "repeat"]["end_time"].max()

In [ ]:
sample_num = df_result[df_result["search_string"] == "repeat"]["end_time"].count()
sample_num

In [ ]:
((df_result[df_result["search_string"] == "repeat"]["end_time"].sum()*2)-sample_num)/60

In [ ]:
df_result.to_excel("Turkish_200_Word_Talk_Time1_Test.xlsx", index=False)

In [ ]:
df_result_select = df_result.head(84)
df_result_select

In [ ]:
id_list = []
start_list = []
end_list = []
for id, start, end in zip(df_result_select["video_id"].to_list(),df_result_select["start_time"].to_list(),df_result_select["end_time"].to_list()):
    id_list.append(str(id))
    start_list.append(str(start))
    end_list.append(str(end))

id_join = ",".join(id_list)
start_join = ",".join(start_list)
end_join = ",".join(end_list)

df_result_for_embedded = pd.DataFrame(data=[[id_join,start_join,end_join]], columns=["id","start_time","end_time"])
df_result_for_embedded

In [ ]:
df_result_for_embedded.to_excel("Turkish_200_Word_Talk_Time1_Join_Test.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Word_Talk_Time*.xlsx")
output_file

In [ ]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

In [ ]:
# used for one search result
df_result = pd.DataFrame()
for sent in df_sentence_link["search_string"]:
    sent_words = word_tokenize(sent)
    for word in sent_words:
        df_link_default_var = df_link_default
        df_var = df_word_link[df_word_link["search_string"] == word]
        df_var.reset_index(drop=True, inplace=True)
        try:
            var_time_diff = (df_var.loc[0,"end_time"] - df_var.loc[0,"start_time"])
            df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + var_time_diff+1.0
            df_result = pd.concat([df_result,df_var], axis=0)
            df_result = pd.concat([df_result,df_link_default_var], axis=0)
        except:
            pass
df_result.reset_index(drop=True, inplace=True)
df_result